

# Import GTFS

In this example, we import a GTFS feed to our model and perform map matching. 

We use data from Coquimbo, a city in La Serena Metropolitan Area in Chile.


.. seealso::
    Several functions, methods, classes and modules are used in this example:

    * :func:`aequilibrae.transit.Transit`
    * :func:`aequilibrae.transit.lib_gtfs.GTFSRouteSystemBuilder`



In [1]:
# Imports
from uuid import uuid4
from os import remove
from os.path import join
from tempfile import gettempdir

import folium
import geopandas as gpd
import pandas as pd

from aequilibrae.project.database_connection import database_connection
from aequilibrae.transit import Transit
from aequilibrae.utils.create_example import create_example

In [2]:
# Let's create an empty project on an arbitrary folder.
fldr = join(gettempdir(), uuid4().hex)

project = create_example(fldr, "coquimbo")

As the Coquimbo example already has a complete GTFS model, we shall remove its public transport
database for the sake of this example.



In [3]:
remove(join(fldr, "public_transport.sqlite"))

Let's import the GTFS feed.



In [4]:
dest_path = join(fldr, "gtfs_coquimbo.zip")

Now we create our Transit object and import the GTFS feed into our model.
This will automatically create a new public transport database.



In [5]:
data = Transit(project)

transit = data.new_gtfs_builder(agency="Lisanco", file_path=dest_path)

To load the data, we must choose one date. We're going to continue with 2016-04-13 but feel free
to experiment with any other available dates. Transit class has a function allowing you to check
dates for the GTFS feed. It should take approximately 2 minutes to load the data.



In [6]:
transit.load_date("2016-04-13")

# Now we execute the map matching to find the real paths.
# Depending on the GTFS size, this process can be really time-consuming.

# transit.set_allow_map_match(True)
# transit.map_match()

# Finally, we save our GTFS into our model.
transit.save_to_disk()

Saving stops (Step: 12/12)                        :  99%|█████████▊| 77/78 [00:02<00:00, 30.37it/s]t/s]

Now we will plot one of the route's patterns we just imported



In [7]:
conn = database_connection("transit")

patterns = pd.read_sql("SELECT pattern_id, ST_AsText(geometry) geom FROM routes;", con=conn)
stops = pd.read_sql("""SELECT stop_id, ST_X(geometry) X, ST_Y(geometry) Y FROM stops""", con=conn)

We turn the patterns and stops DataFrames into GeoDataFrames so we can plot them more easily.



In [8]:
patterns = gpd.GeoDataFrame(patterns, geometry=gpd.GeoSeries.from_wkt(patterns["geom"]), crs=4326)
stops = gpd.GeoDataFrame(stops, geometry=gpd.GeoSeries.from_xy(stops["X"], stops["Y"]), crs=4326)

And plot out data!



In [9]:
map = patterns.explore(color=["#146DB3", "#EB9719"], style_kwds={"weight": 4}, name="links")
map = stops.explore(m=map, color="black", style_kwds={"radius": 2, "fillOpacity": 1.0}, name="stops")

folium.LayerControl().add_to(map)
map

In [10]:
project.close()

This project at /tmp/6a7bbd0a208647619fe1f9aa91f56c1d is already closed
Saving stops (Step: 12/12)                        : 100%|██████████| 78/78 [00:15<00:00, 30.37it/s]